In [ ]:

# 📥 הורדת Firefox ו-Geckodriver בצורה נכונה
!apt-get update
!apt install -y wget unzip
!wget -q https://ftp.mozilla.org/pub/firefox/releases/115.0.2/linux-x86_64/en-US/firefox-115.0.2.tar.bz2
!tar xjf firefox-115.0.2.tar.bz2
!mv firefox /usr/local/firefox
!ln -s /usr/local/firefox/firefox /usr/local/bin/firefox

# 📥 הורדת Geckodriver
!wget -q https://github.com/mozilla/geckodriver/releases/download/v0.36.0/geckodriver-v0.36.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.36.0-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/

!pip install selenium
!pip install tldextract
!pip install bs4


In [ ]:

stage_3_config_dict = {
        "url_of_anchor_upper_tresh":0.6,
        "url_of_anchor_lower_tresh": 0.31,
        "link_count_html_upper_tresh":0.81,
        "link_count_html_lower_tresh": 0.13,
        "request_url_upper_tresh":0.51,
        "iframe_upper_tresh":6,
        "iframe_lower_tresh": 2,
        "js_lower_tresh":2,
        "js_upper_tresh": 6,
        "nlp_upper_tresh":0.008,
        "nlp_lower_tresh":0.003,

    }

    #for java script sus behivor
def get_high_risk_patterns():
         return [
                 r"eval\s*\(",
                 r"new\s+Function\s*\(",
                 r"document\.write\s*\(",
                 r"onmouseover\s*=",
                 r"setTimeout\s*\(\s*['\"]",
             ]
def get_medium_risk_patterns():
     return [
     r"window\.location",
     r"innerHTML\s*=",
     r"onbeforeunload",
 ]
def get_low_risk_patterns():
    return[
     r"navigator\.clipboard",
     r"XMLHttpRequest",
     r"fetch\s*\("
 ]
def get_known_safe_script_hosts():
    return [
     "cdnjs.cloudflare.com",
     "cdn.jsdelivr.net",
     "ajax.googleapis.com",
     "fonts.googleapis.com",
     "fonts.gstatic.com",
     "stackpath.bootstrapcdn.com",
     "ajax.aspnetcdn.com",
     "maxcdn.bootstrapcdn.com",
     "code.jquery.com",
     "cdn.jsdelivr.net",
     "cdn.shopify.com",
     "cdn.wix.com",
     "unpkg.com",
     "polyfill.io",
     "bootstrapcdn.com",
     "gstatic.com",
     "google.com",
     "googleapis.com",
     "microsoft.com",
     "cloudflare.com",
     "cloudfront.net",
     "fbcdn.net",
     "facebook.com",
     "yahooapis.com",
     "notion.so",
     "vercel.app",
     "netlify.app",
     "res.cloudinary.com"
 ]

def get_known_favicon_hosts():
   return ["google.com", "gstatic.com", "googleusercontent.com", "googleapis.com", "youtube.com",
                        "ytimg.com",
                        "apple.com", "microsoft.com", "office.com", "windows.com", "live.com", "microsoftonline.com",
                        "adobe.com", "typekit.net", "adobestatic.com", "facebook.com", "fbcdn.net", "instagram.com",
                        "cdninstagram.com", "twitter.com", "twimg.com",
                        "linkedin.com", "licdn.com", "pinterest.com", "pinimg.com", "reddit.com", "redditstatic.com",
                        "tumblr.com", "static.tumblr.com",
                        "fonts.googleapis.com", "fonts.gstatic.com", "ajax.googleapis.com",
                        "cloudflare.com", "cdnjs.cloudflare.com", "cdn.jsdelivr.net",
                        "cdn.shopify.com", "stackpath.bootstrapcdn.com", "ajax.aspnetcdn.com",
                        "akamaihd.net", "akamaized.net", "fastly.net", "cloudfront.net", "unpkg.com",
                        "raw.githubusercontent.com", "github.com", "github.githubassets.com",
                        "wp.com", "i0.wp.com", "i1.wp.com", "i2.wp.com",
                        "squarespace.com", "squarespace-cdn.com", "static1.squarespace.com", "shopify.com",
                        "cdn.shopify.com", "wix.com", "wixstatic.com",
                        "paypal.com", "paypalobjects.com", "ebay.com", "ebaystatic.com",
                        "amazon.com", "amazonaws.com",
                        "yahoo.com", "yimg.com", "yahooapis.com", "bootstrapcdn.com", "maxcdn.bootstrapcdn.com",
                        "jsdelivr.net", "fastly.com",
                        "googletagmanager.com", "googlesyndication.com", "doubleclick.net", "googledomains.com",
                        "firebaseio.com", "firebaseapp.com", "notion.so", "notion-static.com",
                        "netlify.app", "vercel.app", "cloudinary.com", "res.cloudinary.com"
                        ]
def get_suspicious_keywords():
     return ["login", "signin", "verify", "auth", "password", "2fa", "secure"]

from re import error
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from tldextract import extract
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
import logging
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from multiprocessing import Process

SUSPICIOUS_WORDS_REGEX = re.compile(
    r"(log[\s\-]?in|sign[\s\-]?in|auth|user(name)?|email|phone|account|"
    r"credential|password|passcode|pin|security[\s\-]?code|credit[\s\-]?card|cvv|expiry|iban|bank)",
    re.IGNORECASE
)


lEGIT=1
sUS=0
pHISHING=-1
data = {
    "URL": [
       "https://heatmap.com","https://gamepedia.com"
       ,"https://9xflix.pink","https://redditblog.com"
    ],
    "validity": ['safe', 'safe', 'safe', 'safe']
}

df = pd.DataFrame(data)
df.to_csv("/content/test_urls.csv", index=False)
print("✅ Created small test CSV!")
df.head()

In [ ]:

def element_extraction_from_html(soup_html: BeautifulSoup, tag: str = None, attribute=None) -> list:
    try:
        if not tag:
            return []
        if attribute:
            return soup_html.find_all(tag, **{attribute: True})
        else:
            return soup_html.find_all(tag)
    except Exception as e:
        logging.error(f"[ERROR] Failed to extract elements for tag={tag} attribute={attribute}: {e}")
        return []

def normalize_domain(url: str):
    try:
        parts_of_url = extract(url)
        if not (parts_of_url.domain and parts_of_url.suffix):
            return None
        domain = f"{parts_of_url.domain}.{parts_of_url.suffix}"
        return domain
    except Exception as e:
        logging.error(f"[ERROR] Failed to normalize domain for url={url}: {e}")
        return None

''' specific  function: check the html element that competable with each tag that indicates for a phishing site '''

def safe_extract(tag, attribute):
    try:
        if tag and hasattr(tag, 'get'):
            return tag.get(attribute, "").strip()
        else:
            return ""
    except Exception:
        return ""



#####################1-favicon#############################
def favicon_check(link_tag: list, base_domain: str) -> int:
    try:
        icon_links = []
        for link in link_tag:
            try:
                rel = link.get("rel", [])
                if any("icon" in r.lower() for r in rel):
                    icon_links.append(link)
            except Exception as e:
                logging.error(f"[ERROR] Problem reading link rel attribute: {e}")
                continue

        if not icon_links:
            return lEGIT

        for link in icon_links:
            try:
                href = link.get("href", "")
                parsed = normalize_domain(href)
                if not parsed:
                    continue

                netloc = parsed.lower()
                if any(safe in netloc for safe in get_known_favicon_hosts()):
                    continue

                expected_domain = base_domain.lower()

                if not (netloc == expected_domain):
                    return pHISHING

                if not (href.endswith(".ico") or href.endswith(".png") or href.endswith(".gif")):
                    return pHISHING
            except Exception as e:
                logging.error(f"[ERROR] Problem analyzing favicon link: {e}")
                continue

        return lEGIT
    except Exception as e:
        logging.error(f"[ERROR] Failed in favicon_check: {e}")
        return sUS



#####################2-anchor#############################
def extract_url_of_anchor_feature(a_list: list, base_domain) -> int:
    try:
        total = int(len(a_list))
        phishy_count = 0

        for tag in a_list:
            try:
                href = safe_extract(tag, "href")

                if (not href) or href.strip() in ["#", "javascript:void(0);", "javascript:"]:
                    phishy_count += 1
                    continue

                href_domain = normalize_domain(href)
                if not href_domain:
                    continue
                if href_domain != base_domain:
                    phishy_count += 1
            except Exception as e:
                logging.error(f"[ERROR] Problem analyzing anchor tag: {e}")
                continue

        if total == 0:
            return lEGIT

        ratio = phishy_count / total
        if ratio > stage_3_config_dict["url_of_anchor_upper_tresh"]:
            return pHISHING
        elif stage_3_config_dict["url_of_anchor_lower_tresh"] <= ratio <= stage_3_config_dict["url_of_anchor_upper_tresh"]:
            return sUS
        else:
            return lEGIT
    except Exception as e:
        logging.error(f"[ERROR] Failed in extract_url_of_anchor_feature: {e}")
        return sUS



#####################3-link_count#############################
def link_count_in_html(total_link_list: list, base_domain: str) -> int:
    try:
        total = len(total_link_list)
        if not total:
            return lEGIT

        phishy_count = 0

        for tag in total_link_list:
            try:
                tag_name = tag.name.lower()

                if tag_name == "script":
                    value = safe_extract(tag, "src")
                elif tag_name == "meta":
                    value = safe_extract(tag, "content")
                else:
                    value = safe_extract(tag, "href")

                if not value:
                    continue

                domain = normalize_domain(value)
                if not domain:
                    continue

                if domain != base_domain:
                    phishy_count += 1
            except Exception as e:
                logging.error(f"[ERROR] Problem analyzing link tag: {e}")
                continue

        ratio = phishy_count / total
        if ratio > stage_3_config_dict["link_count_html_upper_tresh"]:
            return pHISHING
        elif stage_3_config_dict["link_count_html_lower_tresh"] < ratio <= stage_3_config_dict["link_count_html_upper_tresh"]:
            return sUS
        else:
            return lEGIT
    except Exception as e:
        logging.error(f"[ERROR] Failed in link_count_in_html: {e}")
        return sUS



#####################4-request_url#############################
def extract_request_url_feature(elements: list, base_domain: str) -> int:
    try:
        total = len(elements)
        if total == 0:
            return lEGIT

        external_count = 0
        for tag in elements:
            try:
                src = safe_extract(tag, "src")
                if not src:
                    continue

                domain = normalize_domain(src)
                if not domain:
                    continue

                if domain != base_domain:
                    external_count += 1
            except Exception as e:
                logging.error(f"[ERROR] Problem analyzing request URL tag: {e}")
                continue

        ratio = external_count / total
        if ratio >= stage_3_config_dict["request_url_upper_tresh"]:
            return pHISHING
        else:
            return lEGIT
    except Exception as e:
        logging.error(f"[ERROR] Failed in extract_request_url_feature: {e}")
        return sUS


#####################5-sfh#############################
def extract_server_form_handler_feature(form_list: list, base_domain: str) -> int:
    try:
        total_score = 0
        for form in form_list:
            form_score = 0
            try:
                action = safe_extract(form, "action")
                if not action or action.lower() in ["about:blank", "#", ""]:
                    form_score += 2
                else:
                    action_domain = normalize_domain(action)
                    if action_domain and action_domain != base_domain:
                        form_score += 4

                inputs = form.find_all("input") if hasattr(form, 'find_all') else []
                for i in inputs:
                    try:
                        t = i.get("type", "").lower()
                        n = i.get("name", "").lower()
                        if t == "password":
                            form_score += 2
                        if any(w in n for w in get_suspicious_keywords()):
                            form_score += 1
                    except Exception as e:
                        logging.error(f"[ERROR] Problem analyzing input field: {e}")
                        continue
            except Exception as e:
                logging.error(f"[ERROR] Problem analyzing form element: {e}")
                continue

            total_score += form_score

        if total_score >= 5:
            return pHISHING
        elif total_score >= 2:
            return sUS
        else:
            return lEGIT

    except Exception as e:
        logging.error(f"[ERROR] Failed in extract_server_form_handler_feature: {e}")
        return sUS



#####################6-iframe#############################
def extract_iframe_feature(iframe_list: list, base_domain: str) -> int:
    if not iframe_list:
        return lEGIT

    total_score = 0

    try:
        for iframe in iframe_list:
            try:
                score = 0
                src = iframe.get("src", "").strip().lower()
                srcdoc = iframe.get("srcdoc", "").strip().lower()
                style = iframe.get("style", "").lower()
                width = iframe.get("width", "").strip()
                height = iframe.get("height", "").strip()
                iframe_domain = normalize_domain(src) if src else ""

                is_external = iframe_domain and iframe_domain != base_domain

                if any(x in src for x in ["ads", "analytics", "pixel", "tracker", "doubleclick"]):
                    continue

                if "display:none" in style or "visibility:hidden" in style:
                    score += 3

                if width == "0" or height == "0":
                    score += 2

                if is_external:
                    score += 2

                if hasattr(iframe, 'has_attr') and not iframe.has_attr("sandbox"):
                    score += 1

                if srcdoc:
                    try:
                        from bs4 import BeautifulSoup
                        clean_srcdoc_text = BeautifulSoup(srcdoc, "html.parser").get_text().lower()

                        if SUSPICIOUS_WORDS_REGEX.search(clean_srcdoc_text):
                            score += 3
                        if "<script" in srcdoc or "javascript:" in srcdoc:
                            score += 3
                        if "display:none" in srcdoc or "visibility:hidden" in srcdoc:
                            score += 2
                    except Exception as e:
                        logging.error(f"[ERROR] Problem parsing srcdoc in iframe: {e}")
                        continue

                total_score += score

            except Exception as e:
                logging.error(f"[ERROR] Problem analyzing iframe tag: {e}")
                continue

        if total_score >= stage_3_config_dict["iframe_upper_tresh"]:
            return pHISHING
        elif stage_3_config_dict["iframe_lower_tresh"] <= total_score < stage_3_config_dict["iframe_upper_tresh"]:
            return sUS
        else:
            return lEGIT

    except Exception as e:
        logging.error(f"[ERROR] Failed in extract_iframe_feature: {e}")
        return sUS


#####################7-suspicious_js#############################
### need to improve this function-js behavior#####!!!!!!!!!!!!!

def detect_suspicious_js_behavior(soup: BeautifulSoup, base_domain: str) -> int:
    score = 0

    try:
        inline_scripts = soup.find_all("script", src=False)
    except Exception as e:
        logging.error(f"[ERROR] Failed in extract_iframe_feature: {e}")
        return sUS

    for script in inline_scripts:
        try:
            content = script.get_text().strip().lower()
        except Exception:
            content=""

        for pattern in get_high_risk_patterns():
            if re.search(pattern, content):
                score += 3

        for pattern in get_medium_risk_patterns():
            if re.search(pattern, content):
                score += 2

        for pattern in get_low_risk_patterns():
            if re.search(pattern, content):
                score += 1

    try:
        external_scripts = soup.find_all("script", src=True)
        for script in external_scripts:
            src = safe_extract(script, "src")
            domain = normalize_domain(src)
            if domain and domain != base_domain and domain not in get_known_safe_script_hosts():
                score += 1

        if score >= stage_3_config_dict["js_upper_tresh"]:
            return pHISHING
        elif score >= stage_3_config_dict["js_lower_tresh"]:
            return sUS
        else:
            return lEGIT

    except Exception as e:
        logging.error(f"[ERROR] Failed in extract_iframe_feature: {e}")
        return sUS




#####################8-nlp#############################
def nlp_based_phishing_text_check(soup: BeautifulSoup) -> int:
    try:
         text = soup.get_text(strip=True).lower()
    except Exception as e:
        logging.error(f"[ERROR] Failed in extract_iframe_feature: {e}")
        return sUS
    matches = SUSPICIOUS_WORDS_REGEX.findall(text)

    if not matches:
        return lEGIT

    total_words = len(text.split())
    match_ratio = len(matches) / total_words if total_words > 0 else 0

    if match_ratio > stage_3_config_dict["nlp_upper_tresh"]:
        return pHISHING
    elif match_ratio > stage_3_config_dict["nlp_lower_tresh"]:
        return sUS
    else:
        return lEGIT



#####################9-analyze_textual_tags#############################
def analyze_textual_tags(soup: BeautifulSoup) -> int:
    try:
        tags = soup.find_all(["meta", "script"])
        texts = []

        for t in tags:
            try:
                content = t.get("content", "")
                string = t.string or ""
                combined = (content + " " + string).strip()
                if combined:
                    texts.append(combined)
            except Exception as e:
                logging.error(f"[ERROR] Problem analyzing meta/script tag: {e}")
                continue

        if not texts:
            return lEGIT

        full_text = " ".join(texts).lower()
        matches = SUSPICIOUS_WORDS_REGEX.findall(full_text)
        total_words = len(full_text.split())

        if total_words == 0:
            return lEGIT

        match_ratio = len(matches) / total_words

        if total_words < 50:
            if match_ratio > 0.05:
                return pHISHING
            elif match_ratio > 0.015:
                return sUS
            else:
                return lEGIT
        else:
            if match_ratio > 0.03:
                return pHISHING
            elif match_ratio > 0.01:
                return sUS
            else:
                return lEGIT

    except Exception as e:
        logging.error(f"[ERROR] Failed in analyze_textual_tags: {e}")
        return sUS





#####################10-dynamic_script#############################
def detect_dynamic_script_injection(driver: webdriver) -> int:
    try:
        injected_scripts = driver.execute_script("""
            return [...document.scripts].filter(s => s.src || s.innerText.length > 0).length;
        """)
        if injected_scripts > 10:
            return pHISHING
        elif injected_scripts > 5:
            return sUS
        return lEGIT
    except Exception as e:
        logging.error(f"[ERROR] Failed in analyze_textual_tags: {e}")
        return sUS



#####################11-auto_redirect#############################
############## after all html was loaded
# this function is blocking because the waiting for full upload of the page ,
def detect_auto_redirect(driver: webdriver, base_domain: str, timeout: float = 3.0) -> int:
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )

        page_source = driver.page_source.lower()

        if re.search(r'<meta\s+http-equiv\s*=\s*["\']?refresh["\']?', page_source, re.IGNORECASE):
            return pHISHING

        if re.search(r'(window\.)?location\.(href|replace)', page_source):
            return pHISHING

        final_url = driver.current_url
        if not final_url:
            return sUS

        if normalize_domain(final_url) != base_domain:
            return pHISHING

        return lEGIT

    except Exception:
        return sUS



#####################12-login_form_visibility#############################
def check_login_form_visibility(driver: webdriver) -> int:
    try:
        script = """
        var forms = document.getElementsByTagName('form');
        for (var i = 0; i < forms.length; i++) {
            var style = window.getComputedStyle(forms[i]);
            if (style.display === 'none' || style.visibility === 'hidden' ||
                forms[i].offsetWidth === 0 || forms[i].offsetHeight === 0) {
                return true;
            }
        }
        return false;
        """
        hidden = driver.execute_script(script)
        return pHISHING if hidden else lEGIT
    except Exception as e:
        logging.error(f"[ERROR] Failed in analyze_textual_tags: {e}")
        return sUS


#####################13-onmouseover#############################
def detect_onmouseover_in_dom(soup: BeautifulSoup) -> int:
    try:
        tags_with_onmouseover = soup.find_all(attrs={"onmouseover": True})

        inline_scripts = soup.find_all("script", src=False)
        suspicious_script = any("onmouseover" in (script.string or "").lower() for script in inline_scripts)

        if tags_with_onmouseover or suspicious_script:
            return pHISHING
        else:
            return lEGIT
    except Exception:
        return sUS



#####################14-right_click_block#############################
def detect_right_click_block(soup: BeautifulSoup) -> int:
    try:
        contextmenu_tags = soup.find_all(attrs={"oncontextmenu": True})
        inline_scripts = soup.find_all("script", src=False)
        suspicious_script = False
        for script in inline_scripts:
            content = (script.string or "").lower()

            if not content:
                continue

            if re.search(r"event\s*\.\s*button\s*==\s*2", content) or "contextmenu" in content:
                suspicious_script = True
                break

        if contextmenu_tags or suspicious_script:
            return pHISHING
        else:
            return lEGIT

    except Exception:
        return sUS

###########################end###########################################


def find_html_features(html:BeautifulSoup, url: str, feature_type: str,driver:webdriver):
    domain = normalize_domain(url)
    str_html = str(html)
    elements=[]
    if feature_type == "favicon_check":
        elements = element_extraction_from_html(html, tag="link", attribute="href")
        return favicon_check(elements, domain)
    elif feature_type == "url_anchor":
        elements = element_extraction_from_html(html, tag="a", attribute="href")
        return extract_url_of_anchor_feature(elements,domain)
    elif feature_type == "links_in_tags":
        elements+= element_extraction_from_html(html, tag="meta", attribute="content")
        elements += element_extraction_from_html(html, tag="script", attribute="src")
        elements += element_extraction_from_html(html, tag="link", attribute="href")
        return link_count_in_html(elements,domain)
    elif feature_type== "request_sources_from_diff_url":
        for tag in ["img","source","audio","video","embed","iframe"]:
            elements+= element_extraction_from_html(html, tag=tag, attribute="src")
        return extract_request_url_feature(elements,domain)
    elif feature_type == "sfh":
        elements = element_extraction_from_html(html, tag="form", attribute="action")
        elements += element_extraction_from_html(html, tag="form")
        return extract_server_form_handler_feature(elements, domain)
    elif feature_type=="iframe":
        elements+=element_extraction_from_html(html,tag="iframe")
        return extract_iframe_feature(elements,domain)
    elif feature_type == "suspicious_js":
        return detect_suspicious_js_behavior(html,domain)
    elif feature_type =="nlp_text":
       return nlp_based_phishing_text_check(html)
    elif feature_type =="analyze_textual_tags":
        return  analyze_textual_tags(html)
    elif feature_type =="detect_dynamic_script_injection":
        return detect_dynamic_script_injection(driver)
    elif feature_type == "detect_auto_redirect":
        return detect_auto_redirect(driver,domain)
    elif feature_type == "check_login_form_visibility":
        return check_login_form_visibility(driver)
    elif feature_type == "detect_onmouseover_in_dom":
        return detect_onmouseover_in_dom(html)
    elif feature_type == "detect_right_click_block":
        return  detect_right_click_block(html)
    return


def wait_for_initial_html(driver, min_length=500, timeout=5.0) -> str:
    """
    Waits until there is enough initial HTML content.
    Returns the HTML once ready, or the best available after timeout.
    """
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            ready_state = driver.execute_script("return document.readyState;")
            if ready_state not in ["interactive", "complete"]:
                time.sleep(0.05)
                continue

            html = driver.execute_script("return document.documentElement.outerHTML;")
            if html and len(html) >= min_length:
                return html
        except Exception:
            pass

        time.sleep(0.05)

    # fallback
    try:
        html = driver.execute_script("return document.documentElement.outerHTML;")
    except Exception:
        html = ""
    return html

def test_headless_browser_firefox(url: str, mode: str = "full") -> tuple:
    """
    mode: "initial" -> fetch initial enough HTML
          "full"    -> wait for fully loaded HTML
    """
    driver = None
    try:

        if not url.startswith(('http://', 'https://')):
            url = 'http://' + url

        ffx_options = Options()
        ffx_options.add_argument("--headless")
        driver = webdriver.Firefox(options=ffx_options)
        driver.get(url)

        if mode == "full":

            # 🔵 מצב FULL → לחכות שהדף יטען לגמרי
            WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")
            html = driver.page_source
        elif mode == "initial":
            # 🟢 מצב INITIAL → לחכות שיהיה מספיק HTML ראשוני
            html = wait_for_initial_html(driver, min_length=500, timeout=5.0)
        else:
            raise ValueError("Unknown mode. Use 'initial' or 'full'.")

        if not html or html.strip() == "":
            logging.error(f"[ERROR] Empty page source for {url}")
            return None, None

        return html, driver

    except Exception as e:
        logging.error(f"❌ [ERROR] Failed to load {url}: {e}")
        return None, None

    finally:
        if driver is not None:
            try:
                driver.quit()
            except Exception:
                pass

# Set logging level to DEBUG
logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(levelname)s | %(message)s')

def extract_stage3_features_debug(input_csv_path, output_csv_path, id):
    driver = None
    results = []
    try:
        df = pd.read_csv(input_csv_path)
        total = len(df)

        for index, row in df.iterrows():
            url = row['URL']
            label = row.get('validity', 0)
            print(f"{id}:\n🔍 [{index+1}/{total}] Processing URL: \033[96m{url}\033[0m")

            html, driver = test_headless_browser_firefox(url,"full")  #change here to "full" or "initial"
            if html is None or driver is None:
                logging.error(f"[ERROR] Failed to load page: {url}")
                continue

            soup = BeautifulSoup(html, "html.parser")
            features = [url]

            for feature_type in [
                "favicon_check",
                "url_anchor",
                "links_in_tags",
                "request_sources_from_diff_url",
                "sfh",
                "iframe",
                "suspicious_js",
                "nlp_text",
                "analyze_textual_tags",
                "detect_dynamic_script_injection",
                "detect_auto_redirect",
                "check_login_form_visibility",
                "detect_onmouseover_in_dom",
                "detect_right_click_block"
            ]:
                try:
                    result = find_html_features(soup, url, feature_type, driver)
                    logging.info(f"{feature_type:35} → {result}")
                    features.append(result)
                except Exception as e:
                    logging.error(f"[ERROR] {feature_type} failed for {url} → {e}")
                    features.append(-999)

            features.append(label)
            results.append(features)

    except Exception as e:
        logging.error(f"[ERROR] General failure in extract_stage3_features_debug: {e}")

    finally:
        if driver is not None:
            try:
                driver.quit()
            except Exception:
                pass

        # Define headers
        headers = [
            "url", "favicon_check", "url_anchor", "links_in_tags",
            "request_sources_from_diff_url", "sfh", "iframe",
            "suspicious_js", "nlp_text", "analyze_textual_tags",
            "detect_dynamic_script_injection", "detect_auto_redirect",
            "check_login_form_visibility", "detect_onmouseover_in_dom",
            "detect_right_click_block", "label"
        ]

        # Save to CSV
        try:
            pd.DataFrame(results, columns=headers).to_csv(output_csv_path, index=False)
            logging.info(f"\n✅ Finished. CSV saved to: \033[92m{output_csv_path}\033[0m")
        except Exception as e:
            logging.error(f"[ERROR] Failed saving CSV {output_csv_path}: {e}")



def process_single_file(input_csv_path, output_csv_path,pid):
    try:
        logging.info(f"🚀 Starting pid= {pid} for {input_csv_path}")
        extract_stage3_features_debug(input_csv_path, output_csv_path,pid)
        logging.info(f"✅ Finished pid={pid} for {input_csv_path}")
    except Exception as e:
        logging.error(f"❌ Error in pid={pid} processing {input_csv_path}: {e}")

def main_parallel():

    files = [
        ("/content/phish_urls_part_1.csv", "/content/phish_urls_part_1.csv","1"),
        ("/content/phish_urls_part_2.csv", "/content/phish_urls_part_2.csv","2"),
        ("/content/phish_urls_part_3.csv", "/content/phish_urls_part_3.csv","3"),
         ("/content/phish_urls_part_4.csv", "/content/phish_urls_part_4.csv","4")
    ]

    total_files = len(files)
    print(f"🚀 Starting sequential processing of {total_files} files...\n")

    for idx, (input_csv, output_csv, id) in enumerate(files, 1):
        print(f"🔵 [{idx}/{total_files}] Processing file ID={id}")
        try:
            process_single_file(input_csv, output_csv, id)
            print(f"✅ Finished processing file ID={id}\n")
        except Exception as e:
            print(f"❌ Error processing file ID={id}: {e}\n")

    print("🎯 All files processed successfully!\n")

main_parallel()
